In [1]:
import json
import rsa
import hashlib
import random

# --- Step 1: Load addresses from JSON file ---
with open("network.json", "r") as file:
    data = json.load(file)

master_address = data["master_address"]
linked_addresses = data["linked_addresses"]

node_A = linked_addresses[0]["address"]  # Sender
node_B = linked_addresses[1]["address"]  # Receiver

# --- Step 2: Message to be sent ---
message = "Patient X has diabetes"
message_hash = int.from_bytes(hashlib.sha256(message.encode()).digest(), byteorder='big')

# --- Step 3: Generate RSA key pair for the signer (master node) ---
(pubkey, privkey) = rsa.newkeys(2048)

# --- Step 4: Generate a secure blinding factor ---
n = pubkey.n
e = pubkey.e
while True:
    r = random.randrange(2, n-1)
    if rsa.common.inverse(r, n):  # Ensure r is invertible
        break
r_inv = rsa.common.inverse(r, n)

# --- Step 5: Node A (Sender) blinds the message hash ---
blinded_msg = (message_hash * pow(r, e, n)) % n

# --- Step 6: Master node signs the blinded message ---
blinded_signature = pow(blinded_msg, privkey.d, n)

# --- Step 7: Node A unblinds the signature ---
unblinded_signature = (blinded_signature * r_inv) % n

# --- Step 8: Node B verifies the signature using master's public key ---
verified_hash = pow(unblinded_signature, e, n)

# --- Output Summary ---
print(f"\n🔐 Blind Signature Simulation")
print(f"Signer (Master Node): {master_address}")
print(f"Sender (Node A):      {node_A}")
print(f"Receiver (Node B):    {node_B}")
print(f"Message:              {message}")
print(f"Is Signature Valid?:  {verified_hash == message_hash}")



🔐 Blind Signature Simulation
Signer (Master Node): 19TVh7M9UhbaVtHFKRszrmMoCfLBJmAc2RcUwa
Sender (Node A):      13xnXAJysKKqk97mSWoRXBF9NJ3u8ZDH8Havgr
Receiver (Node B):    1K1N7zH1ErysKyuFstj6fd63JDKy1CgsCgqPJP
Message:              Patient X has diabetes
Is Signature Valid?:  True
